# Module 2.2: Canny Edge Detection（Canny 邊緣偵測）

## 學習目標

本 notebook 將帶你從零實作完整的 **Canny Edge Detector**：

1. **Gaussian Smoothing**：降低雜訊
2. **Gradient Computation**：計算梯度強度和方向
3. **Non-Maximum Suppression (NMS)**：細化邊緣
4. **Hysteresis Thresholding**：雙閾值連接

## 前置要求
- 完成 Module 1（卷積、濾波器、Sobel）
- 完成 Module 2.1（Otsu）

## 參考資料
- Canny, 1986. "A Computational Approach to Edge Detection"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from collections import deque
import sys
sys.path.append('../../')

try:
    from utils.image_io import load_grayscale, save_image
except:
    def load_grayscale(path):
        return np.array(Image.open(path).convert('L'), dtype=np.float64)
    def save_image(arr, path):
        Image.fromarray(arr.astype(np.uint8)).save(path)

print("環境設定完成！")

## 複習：從 Module 1 引入必要函數

In [ ]:
# 卷積函數（Module 1.1）
def conv2d(image, kernel, padding='same'):
    """2D 卷積"""
    kH, kW = kernel.shape
    
    if padding == 'valid':
        pad_h, pad_w = 0, 0
    elif padding == 'same':
        pad_h = (kH - 1) // 2
        pad_w = (kW - 1) // 2
    else:
        pad_h = pad_w = padding
    
    if pad_h > 0 or pad_w > 0:
        image = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant')
    
    H, W = image.shape
    out_H = H - kH + 1
    out_W = W - kW + 1
    
    output = np.zeros((out_H, out_W), dtype=np.float64)
    
    for i in range(out_H):
        for j in range(out_W):
            output[i, j] = np.sum(image[i:i+kH, j:j+kW] * kernel)
    
    return output


# Gaussian kernel（Module 1.2）
def gaussian_kernel(size, sigma):
    """建立 Gaussian kernel"""
    half = size // 2
    x = np.arange(-half, half + 1)
    X, Y = np.meshgrid(x, x)
    gaussian = np.exp(-(X**2 + Y**2) / (2 * sigma**2))
    return gaussian / gaussian.sum()


def gaussian_blur(image, sigma):
    """Gaussian 平滑"""
    size = int(np.ceil(6 * sigma)) | 1  # 確保是奇數
    kernel = gaussian_kernel(size, sigma)
    return conv2d(image, kernel, padding='same')


# Sobel（Module 1.2）
SOBEL_X = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=np.float64)
SOBEL_Y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=np.float64)

print("基礎函數已載入！")

## 建立測試影像

In [ ]:
def create_test_image(size=150):
    """
    建立包含各種邊緣的測試影像。
    """
    np.random.seed(42)
    image = np.zeros((size, size), dtype=np.float64)
    
    # 背景漸層
    for i in range(size):
        image[i, :] = 80 + i * 0.3
    
    # 大方塊
    image[20:60, 20:60] = 200
    
    # 圓形
    y, x = np.ogrid[:size, :size]
    center_y, center_x = 40, 110
    radius = 25
    mask = (x - center_x)**2 + (y - center_y)**2 <= radius**2
    image[mask] = 180
    
    # 對角線
    for i in range(50):
        if 80 + i < size and 20 + i < size:
            image[80+i, 20+i:25+i] = 220
    
    # 小方塊群
    for i in range(3):
        for j in range(3):
            y0 = 90 + i * 15
            x0 = 90 + j * 15
            if y0 + 10 < size and x0 + 10 < size:
                image[y0:y0+10, x0:x0+10] = 160 + (i + j) * 20
    
    # 加入適量雜訊
    noise = np.random.randn(size, size) * 5
    image = image + noise
    
    return np.clip(image, 0, 255)

# 建立並顯示
test_image = create_test_image(150)

plt.figure(figsize=(8, 8))
plt.imshow(test_image, cmap='gray', vmin=0, vmax=255)
plt.title('Test Image')
plt.colorbar()
plt.show()

---

# Canny Edge Detection 概觀

## 為什麼需要 Canny？

簡單的梯度方法（如 Sobel）有以下問題：
1. 邊緣太粗（多像素寬）
2. 對雜訊敏感
3. 不清楚如何選擇閾值

## Canny 的三個準則

1. **Good Detection**：最小化漏檢和誤檢
2. **Good Localization**：邊緣位置準確
3. **Single Response**：每個邊緣只有一個回應

## 四個步驟

```
Input Image
    ↓
1. Gaussian Smoothing (降噪)
    ↓
2. Gradient Computation (Sobel)
    ↓
3. Non-Maximum Suppression (細化)
    ↓
4. Hysteresis Thresholding (連接)
    ↓
Edge Map
```

---

# Step 1: Gaussian Smoothing

## 為什麼需要平滑？

- 雜訊會產生假邊緣
- 梯度運算會放大雜訊
- Gaussian 平滑可以抑制高頻雜訊

## σ (sigma) 的選擇

- **σ 小**：保留更多細節，但可能有雜訊
- **σ 大**：更平滑，但可能丟失細節
- 常見選擇：1.0 ~ 2.0

In [ ]:
# Step 1: Gaussian Smoothing
def canny_step1_smoothing(image, sigma=1.4):
    """
    Canny Step 1: Gaussian 平滑。
    
    Parameters
    ----------
    image : np.ndarray
        輸入灰階影像
    sigma : float
        Gaussian 標準差
    
    Returns
    -------
    smoothed : np.ndarray
        平滑後的影像
    """
    return gaussian_blur(image, sigma)

# 測試不同 sigma
smoothed_1 = canny_step1_smoothing(test_image, sigma=1.0)
smoothed_14 = canny_step1_smoothing(test_image, sigma=1.4)
smoothed_2 = canny_step1_smoothing(test_image, sigma=2.0)

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

axes[0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Original')

axes[1].imshow(smoothed_1, cmap='gray', vmin=0, vmax=255)
axes[1].set_title('Smoothed (σ=1.0)')

axes[2].imshow(smoothed_14, cmap='gray', vmin=0, vmax=255)
axes[2].set_title('Smoothed (σ=1.4)')

axes[3].imshow(smoothed_2, cmap='gray', vmin=0, vmax=255)
axes[3].set_title('Smoothed (σ=2.0)')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

---

# Step 2: Gradient Computation

## 目標

計算每個像素的：
- **Gradient Magnitude** $|G| = \sqrt{G_x^2 + G_y^2}$
- **Gradient Direction** $\theta = \arctan2(G_y, G_x)$

## 使用 Sobel 運算子

In [ ]:
def canny_step2_gradient(image):
    """
    Canny Step 2: 計算梯度。
    
    Parameters
    ----------
    image : np.ndarray
        平滑後的影像
    
    Returns
    -------
    magnitude : np.ndarray
        梯度強度
    direction : np.ndarray
        梯度方向（弧度，範圍 -π 到 π）
    """
    # 計算 Sobel 梯度
    Gx = conv2d(image, SOBEL_X, padding='same')
    Gy = conv2d(image, SOBEL_Y, padding='same')
    
    # 計算強度
    magnitude = np.sqrt(Gx**2 + Gy**2)
    
    # 計算方向
    direction = np.arctan2(Gy, Gx)
    
    return magnitude, direction

# 測試
smoothed = canny_step1_smoothing(test_image, sigma=1.4)
magnitude, direction = canny_step2_gradient(smoothed)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(smoothed, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Smoothed')
axes[0].axis('off')

im1 = axes[1].imshow(magnitude, cmap='gray')
axes[1].set_title('Gradient Magnitude')
axes[1].axis('off')
plt.colorbar(im1, ax=axes[1])

im2 = axes[2].imshow(direction, cmap='hsv', vmin=-np.pi, vmax=np.pi)
axes[2].set_title('Gradient Direction')
axes[2].axis('off')
plt.colorbar(im2, ax=axes[2])

plt.tight_layout()
plt.show()

print(f"Magnitude 範圍：{magnitude.min():.1f} ~ {magnitude.max():.1f}")
print(f"Direction 範圍：{direction.min():.2f} ~ {direction.max():.2f} (radians)")

---

# Step 3: Non-Maximum Suppression (NMS)

## 目的

將粗邊緣**細化**成單像素寬。

## 原理

對於每個像素：
1. 找出它的梯度方向
2. 沿著梯度方向看兩個鄰居
3. 如果當前像素不是局部最大值，就抑制它

## 方向量化

將連續的梯度方向量化成 4 個離散方向：

| 角度範圍 | 量化方向 | 比較鄰居 |
|----------|----------|----------|
| -22.5° ~ 22.5° 或 157.5° ~ 180° | 0° (水平) | 左、右 |
| 22.5° ~ 67.5° 或 -157.5° ~ -112.5° | 45° | 左下、右上 |
| 67.5° ~ 112.5° 或 -112.5° ~ -67.5° | 90° (垂直) | 上、下 |
| 112.5° ~ 157.5° 或 -67.5° ~ -22.5° | 135° | 左上、右下 |

In [ ]:
# 先用一個小例子理解 NMS
def explain_nms():
    """
    用一個 5x5 的小例子解釋 NMS。
    """
    # 5x5 的 magnitude（模擬一條從左上到右下的邊緣）
    magnitude = np.array([
        [10, 20, 15, 10,  5],
        [15, 30, 25, 15, 10],
        [10, 25, 50, 25, 15],  # 中間是邊緣最強的地方
        [10, 15, 25, 30, 20],
        [ 5, 10, 15, 20, 15]
    ], dtype=np.float64)
    
    # 對應的 direction（假設邊緣沿 45° 方向）
    # 梯度方向垂直於邊緣，所以是 135° (= -45°)
    direction = np.full((5, 5), np.radians(135))  # 135° = 2.36 rad
    
    print("=== NMS 手算範例 ===")
    print("\nMagnitude:")
    print(magnitude)
    print(f"\nDirection: 全部是 135° (約 {np.radians(135):.2f} rad)")
    print("\n135° 方向表示要和 左上/右下 鄰居比較")
    print("\n對於中心像素 (2,2)，magnitude = 50：")
    print(f"  左上鄰居 (1,1) = {magnitude[1,1]}")
    print(f"  右下鄰居 (3,3) = {magnitude[3,3]}")
    print(f"  50 > 30 且 50 > 30，所以保留 50")
    
    return magnitude, direction

mag_example, dir_example = explain_nms()

In [ ]:
def canny_step3_nms(magnitude, direction):
    """
    Canny Step 3: Non-Maximum Suppression。
    
    Parameters
    ----------
    magnitude : np.ndarray
        梯度強度
    direction : np.ndarray
        梯度方向（弧度）
    
    Returns
    -------
    nms : np.ndarray
        NMS 後的邊緣強度（單像素寬）
    
    實作步驟：
    1. 將方向轉換為角度（度）
    2. 量化為 4 個方向
    3. 對每個像素，比較梯度方向上的鄰居
    4. 只保留局部最大值
    """
    H, W = magnitude.shape
    nms = np.zeros_like(magnitude)
    
    # 將弧度轉為角度，並調整到 0-180 範圍
    angle = np.rad2deg(direction)
    angle[angle < 0] += 180  # 把負角度轉成正的
    
    for i in range(1, H - 1):
        for j in range(1, W - 1):
            # 取得當前方向
            a = angle[i, j]
            m = magnitude[i, j]
            
            # 根據方向決定要比較的鄰居
            # 0° (0-22.5 或 157.5-180): 水平方向，比較左右
            if (0 <= a < 22.5) or (157.5 <= a <= 180):
                n1 = magnitude[i, j-1]  # 左
                n2 = magnitude[i, j+1]  # 右
            
            # 45° (22.5-67.5): 對角線，比較右上和左下
            elif 22.5 <= a < 67.5:
                n1 = magnitude[i-1, j+1]  # 右上
                n2 = magnitude[i+1, j-1]  # 左下
            
            # 90° (67.5-112.5): 垂直方向，比較上下
            elif 67.5 <= a < 112.5:
                n1 = magnitude[i-1, j]  # 上
                n2 = magnitude[i+1, j]  # 下
            
            # 135° (112.5-157.5): 對角線，比較左上和右下
            else:  # 112.5 <= a < 157.5
                n1 = magnitude[i-1, j-1]  # 左上
                n2 = magnitude[i+1, j+1]  # 右下
            
            # 如果是局部最大值，保留；否則抑制
            if m >= n1 and m >= n2:
                nms[i, j] = m
            else:
                nms[i, j] = 0
    
    return nms

# 測試 NMS
nms = canny_step3_nms(magnitude, direction)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(smoothed, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Smoothed Image')

axes[1].imshow(magnitude, cmap='gray')
axes[1].set_title('Gradient Magnitude (thick edges)')

axes[2].imshow(nms, cmap='gray')
axes[2].set_title('After NMS (thin edges)')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("觀察：NMS 後的邊緣變細了，只保留單像素寬")

In [ ]:
# 放大觀察 NMS 效果
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 放大方塊邊緣區域
region = slice(15, 65), slice(15, 65)

axes[0].imshow(magnitude[region], cmap='gray')
axes[0].set_title('Magnitude (zoomed)')

axes[1].imshow(nms[region], cmap='gray')
axes[1].set_title('After NMS (zoomed)')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

---

# Step 4: Hysteresis Thresholding（雙閾值）

## 為什麼需要雙閾值？

**單閾值的問題**：
- 閾值太高：丟失弱邊緣，邊緣可能斷開
- 閾值太低：產生太多雜訊邊緣

## 雙閾值解決方案

使用兩個閾值：$T_{low}$ 和 $T_{high}$

1. **Strong Edge**：magnitude > $T_{high}$（一定是邊緣）
2. **Weak Edge**：$T_{low}$ < magnitude ≤ $T_{high}$（可能是邊緣）
3. **Non-Edge**：magnitude ≤ $T_{low}$（不是邊緣）

## 連接規則

- Strong edges 直接保留
- Weak edges 只有在**連接到 strong edge** 時才保留
- 可以使用 BFS/DFS 或迭代擴散實作

In [ ]:
def canny_step4_hysteresis(nms, low_ratio=0.05, high_ratio=0.15):
    """
    Canny Step 4: Hysteresis Thresholding。
    
    Parameters
    ----------
    nms : np.ndarray
        NMS 後的邊緣強度
    low_ratio : float
        低閾值（相對於最大值的比例）
    high_ratio : float
        高閾值（相對於最大值的比例）
    
    Returns
    -------
    edges : np.ndarray
        二值化邊緣圖（0 或 255）
    
    實作步驟：
    1. 計算 low 和 high 閾值
    2. 標記 strong edges 和 weak edges
    3. 使用 BFS 從 strong edges 擴散，連接 weak edges
    """
    H, W = nms.shape
    
    # 計算閾值
    max_val = nms.max()
    low_thresh = max_val * low_ratio
    high_thresh = max_val * high_ratio
    
    # 標記 strong 和 weak edges
    strong = nms > high_thresh
    weak = (nms > low_thresh) & (nms <= high_thresh)
    
    # 初始化結果：先把 strong edges 標記
    edges = np.zeros((H, W), dtype=np.uint8)
    edges[strong] = 255
    
    # 使用 BFS 從 strong edges 擴散到連接的 weak edges
    # 8-連通鄰域
    neighbors = [(-1, -1), (-1, 0), (-1, 1),
                 (0, -1),           (0, 1),
                 (1, -1),  (1, 0),  (1, 1)]
    
    # 把所有 strong edge 位置加入 queue
    queue = deque()
    strong_y, strong_x = np.where(strong)
    for y, x in zip(strong_y, strong_x):
        queue.append((y, x))
    
    # BFS 擴散
    while queue:
        y, x = queue.popleft()
        
        for dy, dx in neighbors:
            ny, nx = y + dy, x + dx
            
            # 檢查邊界
            if 0 <= ny < H and 0 <= nx < W:
                # 如果鄰居是 weak edge 且還沒被標記
                if weak[ny, nx] and edges[ny, nx] == 0:
                    edges[ny, nx] = 255
                    queue.append((ny, nx))
    
    return edges, low_thresh, high_thresh

# 測試
edges, low_t, high_t = canny_step4_hysteresis(nms, low_ratio=0.05, high_ratio=0.15)

print(f"Low threshold: {low_t:.1f}")
print(f"High threshold: {high_t:.1f}")

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(nms, cmap='gray')
axes[0].set_title('NMS Result')

# 顯示 strong 和 weak edges
strong = nms > high_t
weak = (nms > low_t) & (nms <= high_t)
display = np.zeros((*nms.shape, 3), dtype=np.uint8)
display[strong] = [0, 255, 0]  # 綠色 = strong
display[weak] = [255, 0, 0]    # 紅色 = weak

axes[1].imshow(display)
axes[1].set_title('Strong (green) vs Weak (red) Edges')

axes[2].imshow(edges, cmap='gray')
axes[2].set_title('Final Edges (after hysteresis)')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

---

# 完整 Canny Edge Detector

In [ ]:
def canny_edge_detector(image, sigma=1.4, low_ratio=0.05, high_ratio=0.15):
    """
    完整的 Canny Edge Detector。
    
    Parameters
    ----------
    image : np.ndarray
        輸入灰階影像
    sigma : float
        Gaussian 平滑的標準差
    low_ratio : float
        低閾值比例（相對於最大梯度）
    high_ratio : float
        高閾值比例
    
    Returns
    -------
    edges : np.ndarray
        二值化邊緣圖 (0 或 255)
    
    步驟：
    1. Gaussian Smoothing
    2. Gradient Computation (Sobel)
    3. Non-Maximum Suppression
    4. Hysteresis Thresholding
    """
    # Step 1: Gaussian Smoothing
    smoothed = canny_step1_smoothing(image, sigma)
    
    # Step 2: Gradient Computation
    magnitude, direction = canny_step2_gradient(smoothed)
    
    # Step 3: Non-Maximum Suppression
    nms = canny_step3_nms(magnitude, direction)
    
    # Step 4: Hysteresis Thresholding
    edges, _, _ = canny_step4_hysteresis(nms, low_ratio, high_ratio)
    
    return edges

# 測試完整的 Canny
canny_edges = canny_edge_detector(test_image, sigma=1.4, low_ratio=0.05, high_ratio=0.15)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Original Image')

axes[1].imshow(canny_edges, cmap='gray')
axes[1].set_title('Canny Edges')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# 視覺化完整流程
def visualize_canny_pipeline(image, sigma=1.4, low_ratio=0.05, high_ratio=0.15):
    """
    視覺化 Canny 的每個步驟。
    """
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    # 原圖
    axes[0, 0].imshow(image, cmap='gray', vmin=0, vmax=255)
    axes[0, 0].set_title('1. Original')
    
    # Step 1: Smoothing
    smoothed = canny_step1_smoothing(image, sigma)
    axes[0, 1].imshow(smoothed, cmap='gray', vmin=0, vmax=255)
    axes[0, 1].set_title(f'2. Gaussian Smoothing (σ={sigma})')
    
    # Step 2: Gradient
    magnitude, direction = canny_step2_gradient(smoothed)
    axes[0, 2].imshow(magnitude, cmap='gray')
    axes[0, 2].set_title('3. Gradient Magnitude')
    
    # Step 3: NMS
    nms = canny_step3_nms(magnitude, direction)
    axes[1, 0].imshow(nms, cmap='gray')
    axes[1, 0].set_title('4. Non-Maximum Suppression')
    
    # Step 4: Strong/Weak visualization
    edges, low_t, high_t = canny_step4_hysteresis(nms, low_ratio, high_ratio)
    
    strong = nms > high_t
    weak = (nms > low_t) & (nms <= high_t)
    display = np.zeros((*nms.shape, 3), dtype=np.uint8)
    display[strong] = [0, 255, 0]
    display[weak] = [255, 0, 0]
    
    axes[1, 1].imshow(display)
    axes[1, 1].set_title('5. Strong (G) / Weak (R)')
    
    # Final result
    axes[1, 2].imshow(edges, cmap='gray')
    axes[1, 2].set_title('6. Final Edges')
    
    for ax in axes.flat:
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

visualize_canny_pipeline(test_image)

---

# Part 5: 參數調整

## 三個重要參數

1. **sigma**：Gaussian 平滑程度
2. **low_ratio**：低閾值比例
3. **high_ratio**：高閾值比例

In [ ]:
# 測試不同的 sigma
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

sigmas = [0.5, 1.0, 1.5, 2.5]
for ax, sigma in zip(axes, sigmas):
    edges = canny_edge_detector(test_image, sigma=sigma, low_ratio=0.05, high_ratio=0.15)
    ax.imshow(edges, cmap='gray')
    ax.set_title(f'σ = {sigma}')
    ax.axis('off')

plt.suptitle('Effect of Sigma (Gaussian smoothing)')
plt.tight_layout()
plt.show()

print("觀察：")
print("- σ 小：保留更多細節，但可能有雜訊")
print("- σ 大：更平滑，但細節丟失")

In [ ]:
# 測試不同的閾值
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

threshold_pairs = [
    (0.03, 0.08),
    (0.05, 0.15),
    (0.10, 0.25),
    (0.05, 0.10),
    (0.05, 0.20),
    (0.05, 0.30),
]

for ax, (low, high) in zip(axes.flat, threshold_pairs):
    edges = canny_edge_detector(test_image, sigma=1.4, low_ratio=low, high_ratio=high)
    ax.imshow(edges, cmap='gray')
    ax.set_title(f'low={low}, high={high}')
    ax.axis('off')

plt.suptitle('Effect of Thresholds')
plt.tight_layout()
plt.show()

print("觀察：")
print("- 閾值太低：太多雜訊邊緣")
print("- 閾值太高：邊緣斷裂")
print("- high/low 比例影響連接效果")

---

# Part 6: 練習題

## 練習 1: 自動閾值選擇

In [ ]:
def canny_auto_threshold(image, sigma=1.4, median_factor=0.33):
    """
    使用 median 值自動選擇閾值的 Canny。
    
    一種常見的啟發式方法：
    - low = (1 - median_factor) * median
    - high = (1 + median_factor) * median
    
    Parameters
    ----------
    image : np.ndarray
        輸入影像
    sigma : float
        Gaussian sigma
    median_factor : float
        控制閾值寬度
    
    Returns
    -------
    edges : np.ndarray
        邊緣圖
    """
    # Step 1-3
    smoothed = canny_step1_smoothing(image, sigma)
    magnitude, direction = canny_step2_gradient(smoothed)
    nms = canny_step3_nms(magnitude, direction)
    
    # 計算 median（只看非零值）
    nonzero = nms[nms > 0]
    if len(nonzero) == 0:
        return np.zeros_like(image, dtype=np.uint8)
    
    median_val = np.median(nonzero)
    
    # 計算閾值
    low_thresh = max(0, (1 - median_factor) * median_val)
    high_thresh = (1 + median_factor) * median_val
    
    # 轉換為比例
    max_val = nms.max()
    if max_val > 0:
        low_ratio = low_thresh / max_val
        high_ratio = high_thresh / max_val
    else:
        low_ratio, high_ratio = 0.05, 0.15
    
    # Step 4
    edges, _, _ = canny_step4_hysteresis(nms, low_ratio, high_ratio)
    
    return edges

# 測試自動閾值
auto_edges = canny_auto_threshold(test_image, sigma=1.4)
manual_edges = canny_edge_detector(test_image, sigma=1.4)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Original')

axes[1].imshow(manual_edges, cmap='gray')
axes[1].set_title('Manual Thresholds')

axes[2].imshow(auto_edges, cmap='gray')
axes[2].set_title('Auto Thresholds (median-based)')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

## 練習 2: 比較 Sobel 和 Canny

In [ ]:
def simple_sobel_edge(image, threshold=50):
    """簡單的 Sobel 邊緣偵測（用於比較）"""
    Gx = conv2d(image, SOBEL_X, padding='same')
    Gy = conv2d(image, SOBEL_Y, padding='same')
    magnitude = np.sqrt(Gx**2 + Gy**2)
    
    # 簡單閾值
    edges = (magnitude > threshold).astype(np.uint8) * 255
    return edges

# 比較
sobel_edges = simple_sobel_edge(test_image, threshold=80)
canny_edges = canny_edge_detector(test_image, sigma=1.4)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Original')

axes[1].imshow(sobel_edges, cmap='gray')
axes[1].set_title('Simple Sobel (threshold=80)')

axes[2].imshow(canny_edges, cmap='gray')
axes[2].set_title('Canny')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

print("比較：")
print("1. Sobel：邊緣較粗，可能有雜訊")
print("2. Canny：邊緣細（單像素），雜訊少，邊緣連續")

## 練習 3: 完整的邊緣偵測類別

In [ ]:
class EdgeDetector:
    """
    邊緣偵測工具類別。
    """
    
    SOBEL_X = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=np.float64)
    SOBEL_Y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=np.float64)
    
    @classmethod
    def sobel(cls, image, threshold=None):
        """
        Sobel 邊緣偵測。
        
        Returns gradient magnitude, or binary edges if threshold is given.
        """
        Gx = conv2d(image, cls.SOBEL_X, padding='same')
        Gy = conv2d(image, cls.SOBEL_Y, padding='same')
        magnitude = np.sqrt(Gx**2 + Gy**2)
        
        if threshold is not None:
            return (magnitude > threshold).astype(np.uint8) * 255
        return magnitude
    
    @classmethod
    def canny(cls, image, sigma=1.4, low_ratio=0.05, high_ratio=0.15, auto=False):
        """
        Canny 邊緣偵測。
        """
        if auto:
            return canny_auto_threshold(image, sigma)
        return canny_edge_detector(image, sigma, low_ratio, high_ratio)
    
    @classmethod
    def laplacian(cls, image, threshold=None):
        """
        Laplacian 邊緣偵測（零交叉）。
        """
        kernel = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]], dtype=np.float64)
        laplacian = conv2d(image, kernel, padding='same')
        
        if threshold is not None:
            return (np.abs(laplacian) > threshold).astype(np.uint8) * 255
        return laplacian

# 測試
ed = EdgeDetector

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

axes[0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Original')

axes[1].imshow(ed.sobel(test_image, threshold=80), cmap='gray')
axes[1].set_title('Sobel')

axes[2].imshow(ed.laplacian(test_image, threshold=30), cmap='gray')
axes[2].set_title('Laplacian')

axes[3].imshow(ed.canny(test_image, auto=True), cmap='gray')
axes[3].set_title('Canny (auto)')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

---

# 總結

## Canny Edge Detection 核心概念

| 步驟 | 目的 | 方法 |
|------|------|------|
| 1. Smoothing | 降雜訊 | Gaussian Filter |
| 2. Gradient | 找邊緣候選 | Sobel |
| 3. NMS | 細化邊緣 | 沿梯度方向比較 |
| 4. Hysteresis | 連接邊緣 | 雙閾值 + BFS |

## 關鍵函數

- `canny_step1_smoothing()`: Gaussian 平滑
- `canny_step2_gradient()`: 計算梯度
- `canny_step3_nms()`: 非極大值抑制
- `canny_step4_hysteresis()`: 雙閾值連接
- `canny_edge_detector()`: 完整 Canny

## 參數建議

- **sigma**: 1.0 ~ 2.0（根據雜訊程度調整）
- **low/high ratio**: 建議 high = 2~3 × low

## 下一步

在 Module 3 中，我們將學習**特徵描述子**（Harris Corner, SIFT, HOG），它們使用梯度資訊來描述影像中的關鍵點。

In [ ]:
# 最終驗證
print("=== 函數驗證 ===")

test_funcs = [
    ("conv2d(image, kernel)", lambda: conv2d(test_image, SOBEL_X)),
    ("gaussian_blur(image, sigma)", lambda: gaussian_blur(test_image, 1.4)),
    ("canny_step1_smoothing(image, sigma)", lambda: canny_step1_smoothing(test_image, 1.4)),
    ("canny_step2_gradient(image)", lambda: canny_step2_gradient(smoothed)),
    ("canny_step3_nms(magnitude, direction)", lambda: canny_step3_nms(magnitude, direction)),
    ("canny_step4_hysteresis(nms)", lambda: canny_step4_hysteresis(nms)),
    ("canny_edge_detector(image)", lambda: canny_edge_detector(test_image)),
    ("canny_auto_threshold(image)", lambda: canny_auto_threshold(test_image)),
    ("EdgeDetector.sobel(image)", lambda: EdgeDetector.sobel(test_image)),
    ("EdgeDetector.canny(image)", lambda: EdgeDetector.canny(test_image)),
    ("EdgeDetector.laplacian(image)", lambda: EdgeDetector.laplacian(test_image)),
]

for name, func in test_funcs:
    try:
        result = func()
        print(f"✓ {name}")
    except Exception as e:
        print(f"✗ {name}: {e}")

print("\n所有函數驗證完成！")